<a href="https://colab.research.google.com/github/250183037-cloud/Zhailau-Yernurkemengger/blob/main/SDS_105_Project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# PART 1: SCRAPING – Download raw data and save CSV (Zhailau Yernurkemengger-Kemengerus)

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

BASE_URL = "https://worldpopulationreview.com/us-cities/"
STATES = [
    "alabama","alaska","arizona","arkansas","california","colorado","connecticut",
    "delaware","district-of-columbia","florida","georgia","hawaii","idaho","illinois",
    "indiana","iowa","kansas","kentucky","louisiana","maine","maryland","massachusetts",
    "michigan","minnesota","mississippi","missouri","montana","nebraska","nevada",
    "new-hampshire","new-jersey","new-mexico","new-york","north-carolina","north-dakota",
    "ohio","oklahoma","oregon","pennsylvania","rhode-island","south-carolina",
    "south-dakota","tennessee","texas","utah","vermont","virginia","washington",
    "west-virginia","wisconsin","wyoming"
]

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

all_rows = []

for state in STATES:
    url = BASE_URL + state
    print(f"Scraping state: {state} --> {url}")

    response = requests.get(url, headers=HEADERS)
    response.raise_for_status()

    soup = BeautifulSoup(response.text, "html.parser")
    table = soup.find("table")

    if table is None:
        print(f"WARNING: No table found for state: {state}")
        continue

    rows = table.find_all("tr")

    # Headers
    header_cells = rows[0].find_all(["th", "td"])
    headers_row = [cell.get_text(strip=True) for cell in header_cells]

    # Data rows
    for tr in rows[1:]:
        cells = tr.find_all(["td", "th"])
        row_values = [cell.get_text(strip=True) for cell in cells]

        if len(row_values) != len(headers_row):
            continue

        row_dict = dict(zip(headers_row, row_values))
        row_dict["State"] = state  # add the state
        all_rows.append(row_dict)

    time.sleep(0.2)  # pause between requests

# Create raw DataFrame and save as CSV
df_raw = pd.DataFrame(all_rows)
raw_file = "us_cities_raw.csv"
df_raw.to_csv(raw_file, index=False, encoding="utf-8-sig")

print(f"\n[SCRAPING DONE] Raw data saved to: {raw_file}")
print("Columns in raw dataset:", list(df_raw.columns))
print(df_raw.head())


In [ ]:
# PART 2: CLEANING + STYLED TABLE (Berikkyzy Balnur)

import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import HTML, display

pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

#  reading raw data
raw_file = "us_cities_raw.csv"
df = pd.read_csv(raw_file)

print("\n[READ RAW DATA]")
print("Columns:", list(df.columns))
print(df.head().to_string(index=False))

# helper for numbers
def clean_num(value: str):
    if not isinstance(value, str):
        value = str(value)
    value = value.replace(",", "")
    value = value.replace("%", "")
    return pd.to_numeric(value, errors="coerce")

# Fix broken header
rename_map = {
    "2025 Pop.â\x80\x94": "2025 Pop.",
    "2025 Pop.â\x80\x93": "2025 Pop.",
    "2025 Pop.â": "2025 Pop.",
    "2025 Pop.ã€–": "2025 Pop.",
    "2025 Pop.â": "2025 Pop.",
    "2025 Pop.": "2025 Pop.",

    "2024 Pop.â\x80\x94": "2024 Pop.",
    "2024 Pop.â\x80\x93": "2024 Pop.",
    "2024 Pop.â": "2024 Pop.",
    "2024 Pop.â": "2024 Pop.",
    "2024 Pop.": "2024 Pop.",

    "Area (miÂ²)": "Area (mi²)",
    "Area (miÃ‚Â²)": "Area (mi²)",
    "Area (miÂ)": "Area (mi²)",

    "Annual Changeâ\x80\x93": "Annual Change",
    "Annual Changeâ": "Annual Change",
    "Annual Changeâ": "Annual Change",
}
df.rename(columns=rename_map, inplace=True)

df.columns = df.columns.str.encode('latin1', 'ignore').str.decode('utf-8', 'ignore')
df.columns = df.columns.str.replace(r"[^\x00-\x7F]+", "", regex=True)

# unification of column names
col_unify = {}
for c in df.columns:
    lc = c.lower()
    if "city" == lc:
        col_unify[c] = "City"
    elif "state" == lc:
        col_unify[c] = "State"
    elif "rank" in lc:
        col_unify[c] = "Rank"
    elif "2025" in lc and "pop" in lc:
        col_unify[c] = "Pop2025"
    elif "2024" in lc and "pop" in lc:
        col_unify[c] = "Pop2024"
    elif "annual" in lc and "change" in lc:
        col_unify[c] = "AnnualChange"
    elif "density" in lc:
        col_unify[c] = "Density"
    elif "area" in lc:
        col_unify[c] = "AreaMi2"
    elif "type" in lc:
        col_unify[c] = "Type"

df.rename(columns=col_unify, inplace=True)

# nice state name
df["State"] = (
    df["State"]
    .str.replace("-", " ")
    .str.title()
)

# converting numeric columns
numeric_like_columns = []
for col in df.columns:
    sample_text = "".join(df[col].astype(str).head(10).tolist())
    if any(ch.isdigit() for ch in sample_text):
        numeric_like_columns.append(col)

for col in numeric_like_columns:
    df[col] = df[col].astype(str).apply(clean_num)

print("\n[CLEANED DATA INFO]")
print(df.info())
print(df.head().to_string(index=False))

#  basic stats
print("\n=== Descriptive statistics (describe) ===")
print(df.describe(include="all"))
print("\n=== Missing values per column ===")
print(df.isna().sum())
print("\nNumber of duplicated rows:", df.duplicated().sum())

# можно сохранить уже очищенный вариант отдельно
clean_file = "ny_cities_clean.csv"
df.to_csv(clean_file, index=False, encoding="utf-8-sig")
print(f"\nCleaned dataset saved to: {clean_file}")

# STYLED TABLE

pretty_columns_order = [
    "City",
    "State",
    "Rank",
    "Pop2025",
    "Pop2024",
    "AnnualChange",
    "Density",
    "AreaMi2",
    "Type"
]
pretty_columns_order = [c for c in pretty_columns_order if c in df.columns]
df_display = df[pretty_columns_order].copy()

def highlight_change(val):
    try:
        v = float(val)
    except:
        return ""
    if v > 0:
        return "color: green; font-weight: bold;"
    elif v < 0:
        return "color: red; font-weight: bold;"
    else:
        return ""

pop_cols  = [c for c in df_display.columns if "Pop20" in c]
dens_cols = ["Density"] if "Density" in df_display.columns else []

styled_C = (
    df_display.style
    .set_table_styles(
        [
            {"selector": "th",
             "props": [
                 ("background-color", "#333333"),
                 ("color", "white"),
                 ("text-align", "center"),
                 ("font-weight", "bold"),
                 ("padding", "6px")
             ]},
            {"selector": "th.row_heading, td.row_heading",
             "props": [
                 ("background-color", "#333333"),
                 ("color", "white"),
                 ("font-weight", "bold"),
                 ("border-right", "1px solid #ffffff")
             ]}
        ]
    )
    .set_properties(**{
        "font-family": "Calibri, Arial, sans-serif",
        "font-size": "18px",
        "border": "1px solid #dddddd",
        "padding": "3px 5px"
    })
    .background_gradient(cmap="Blues", subset=pop_cols)
    .background_gradient(cmap="Greens", subset=dens_cols)
    .applymap(highlight_change, subset=["AnnualChange"])
    .format({
        "Rank": "{:,.0f}".format,
        "Pop2025": "{:,.0f}".format,
        "Pop2024": "{:,.0f}".format,
        "Density": "{:,.0f}".format,
        "AreaMi2": "{:,.1f}".format,
        "AnnualChange": "{:+.2f}%".format
    })
)

title_html = """
<h2 style="font-family: Calibri, Arial, sans-serif; margin-bottom: 4px; color:#333;">
    New York Cities by Population 2025 (Scraped from WorldPopulationReview)
</h2>
<p style="font-family: Calibri, Arial, sans-serif; margin-top: 0; font-size: 12px; color: #555;">
    Dataset built using Python, requests, BeautifulSoup, and pandas (state of New York).
</p>
"""

print("\n=== STYLED TABLE PREVIEW ===")
display(HTML(title_html))
display(styled_C)


In [ ]:
# PART 3 — Exploratory Data Analysis (EDA)

import matplotlib.pyplot as plt
import seaborn as sns

eda = df.copy()

# 1) DESCRIPTIVE STATISTICS (Shaimukhamedova Madina)

print("\n======= DESCRIPTIVE STATISTICS =======")
print(eda.describe(include='all'))

print("\nMissing values:")
print(eda.isna().sum())

print("\nCorrelation matrix (numeric features):")
print(eda.corr(numeric_only=True))

# 2) VISUALIZATIONS (6+ meaningful plots) (Shaimukhamedova Madina)


eda_df = df.copy()

# 1) Top 15 fastest-growing cities (by AnnualChange)
growing = eda_df.sort_values("AnnualChange", ascending=False).head(15)

plt.figure(figsize=(10,5))
plt.barh(growing["City"], growing["AnnualChange"], color="green")
plt.xlabel("Annual Change (%)")
plt.title("Top 15 Fastest-Growing US Cities")
plt.gca().invert_yaxis()  # biggest on top
plt.tight_layout()
plt.show()

# 2) Top 15 fastest-shrinking cities (negative AnnualChange)
shrinking = eda_df.sort_values("AnnualChange").head(15)

plt.figure(figsize=(10,5))
plt.barh(shrinking["City"], shrinking["AnnualChange"], color="red")
plt.xlabel("Annual Change (%)")
plt.title("Top 15 Fastest-Shrinking US Cities")
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

# 3) Top 10 largest cities by 2025 population (bar chart)
top10 = eda_df.sort_values("Pop2025", ascending=False).head(10)

plt.figure()
plt.bar(top10["City"], top10["Pop2025"])
plt.xticks(rotation=45, ha="right")
plt.ylabel("Population 2025")
plt.title("Top 10 Largest US Cities by Population (2025)")
plt.tight_layout()
plt.show()

# 4) Scatter: Density vs Population
plt.figure()
plt.scatter(eda_df["Pop2025"], eda_df["Density"], s=5)
plt.xlabel("Population 2025")
plt.ylabel("Population Density (people per mi²)")
plt.title("City Population vs. Density - All States")
plt.tight_layout()
plt.show()

# 5) BAR + LINE PLOT — Select top 15 cities by 2025 population
temp = df.copy()
temp["Delta"] = temp["Pop2025"] - temp["Pop2024"]
top_delta = temp.sort_values("Delta", ascending=False).head(15)

plt.figure(figsize=(10,6))
plt.plot(top_delta["City"], top_delta["Pop2024"], marker="o", label="2024")
plt.plot(top_delta["City"], top_delta["Pop2025"], marker="o", label="2025")
plt.xticks(rotation=45, ha="right")
plt.ylabel("Population")
plt.title("Population Change 2024 → 2025 (Top 15 Cities by Increase)")
plt.legend()
plt.tight_layout()
plt.show()


# 6) PIE CHART — Share of Top 10 Cities by Population (2025)
top10 = df.sort_values("Pop2025", ascending=False).head(10)

plt.figure(figsize=(8, 8))
plt.pie(
    top10["Pop2025"],
    labels=top10["City"],
    autopct="%1.1f%%",
    startangle=140,
    textprops={'fontsize': 10}
)
plt.title("Share of Top 10 Most Populated Cities in New York (2025)")
plt.tight_layout()
plt.show()




In [ ]:
# 3) ANALYSIS QUESTIONS (Khabdolda Akhsyelyeu)

print("\n======= ANALYSIS QUESTIONS =======")

# Q1) Cities with highest/highest population?
print("\nQ1: Largest and smallest cities by 2025 population:")
print("Largest:")
print(eda.sort_values("Pop2025", ascending=False).head(5)[["City","State","Pop2025"]])
print("Smallest:")
print(eda.sort_values("Pop2025").head(5)[["City","State","Pop2025"]])

# Q2) Which state dominates by number of cities?
print("\nQ2: State with the most cities:")
print(eda["State"].value_counts().head(5))

# Q3) Which state has the highest average city population?
print("\nQ3: States with highest average city population:")
print(eda.groupby("State")["Pop2025"].mean().sort_values(ascending=False).head(5))

# Q4) Fastest-growing cities
print("\nQ4: Top 5 fastest-growing cities:")
print(eda.sort_values("AnnualChange", ascending=False).head(5)[["City","State","AnnualChange"]])

# Q5) Fastest-shrinking cities
print("\nQ5: Top 5 shrinking cities:")
print(eda.sort_values("AnnualChange").head(5)[["City","State","AnnualChange"]])

# Q6) Do bigger cities grow faster? (Pop2025 vs AnnualChange correlation)
print("\nQ6: Correlation between population and growth rate:")
print(eda["Pop2025"].corr(eda["AnnualChange"]))

# Q7) Relationship between population and density
print("\nQ7: Correlation between population and density:")
print(eda["Pop2025"].corr(eda["Density"]))

# Q8) Which states have the densest cities?
print("\nQ8: States with highest average density:")
print(eda.groupby("State")["Density"].mean().sort_values(ascending=False).head(5))